## 153: Inicio de sesión de usuario con verificación de cuenta (FastAPI + JWT)

Implementar el endpoint de inicio de sesión (/login) que permita autenticar a un usuario solo si su cuenta ya fue verificada mediante el token de activación. El sistema retornará un JWT de acceso válido por un tiempo determinado.

🧩 Código completo del inicio de sesión con verificación de cuenta
python
Copiar
Editar
from fastapi import APIRouter, HTTPException, Depends
from sqlalchemy.orm import Session
from passlib.context import CryptContext
import jwt
from datetime import datetime, timedelta

router = APIRouter()

SECRET_KEY = "CLAVE_SECRETA_SEGURA"
ALGORITHM = "HS256"
ACCESS_TOKEN_EXPIRE_MINUTES = 30

pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

@router.post("/login")
def login_user(email: str, password: str, db: Session = Depends(get_db)):
    user = db.query(User).filter_by(email=email).first()

    if not user:
        raise HTTPException(status_code=404, detail="❌ Usuario no encontrado")

    if not user.is_active:
        raise HTTPException(status_code=403, detail="⚠️ Cuenta no verificada")

    if not pwd_context.verify(password, user.hashed_password):
        raise HTTPException(status_code=401, detail="🔐 Contraseña incorrecta")

    expiration = datetime.utcnow() + timedelta(minutes=ACCESS_TOKEN_EXPIRE_MINUTES)
    payload = {
        "sub": user.email,
        "exp": expiration
    }
    token = jwt.encode(payload, SECRET_KEY, algorithm=ALGORITHM)

    return {
        "access_token": token,
        "token_type": "bearer"
    }